<a href="https://colab.research.google.com/github/jam2205/500-AI-Agents-Projects/blob/main/Tinymodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
hf_token = userdata.get('HF_TOKEN')

model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
import torch
# Example of generating text
prompt = "What is the capital of France?"
inputs = tokenizer(prompt, return_tensors="pt")

# Determine the device and move the model and inputs to it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the determined device
if model.device.type != device.type:
    model.to(device)

# Move inputs to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model.generate(**inputs, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Prompt: {prompt}")
print(f"Response: {response}")

Prompt: What is the capital of France?
Response: What is the capital of France?


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer (your TinyLlama or similar)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Or your loaded model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Fix: Set pad_token to eos_token if not set (common for chat models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Pad token set to EOS token.")

# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
).to(device)
print(f"✅ Model loaded on {device}. Pad token ID: {tokenizer.pad_token_id}")

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Model loaded on cuda. Pad token ID: 2


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check if the model is loaded before moving it to the device
if model is not None:
    model.to(device)
else:
    print("Error: Model was not loaded successfully in the previous cell.")

**Note:** Be careful when displaying secrets directly in your notebook output, especially if you are sharing it. For security, you might just want to check if the token exists without printing its value.

In [ ]:
from google.colab import userdata

try:
  hf_token = userdata.get('HF_TOKEN') # Assuming your Hugging Face token is stored as 'HF_TOKEN'
  if hf_token:
    print("Hugging Face token found in secrets.")
    # You can optionally print the token value for verification (be cautious):
    # print(f"Token: {hf_token}")
  else:
    print("Hugging Face token not found in secrets. Please add it using the key icon in the left sidebar.")
except Exception as e:
  print(f"An error occurred while accessing secrets: {e}")

Hugging Face token found in secrets.


In [ ]:
def generate_response(messages, max_new_tokens=150, temperature=0.7):
    """
    Generate a response using the chat template.
    messages: List of dicts like [{"role": "user", "content": "Hi"}]
    """
    # Format with chat template (adds <s>[INST] etc. for TinyLlama)
    formatted_input = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True  # Adds </s> for assistant turn
    )
    # Move the input tensor to the same device as the model
    inputs = formatted_input.to(device)

    # Generate (no gradients for inference)
    with torch.no_grad():
        outputs = model.generate(
            inputs=inputs, # Pass the input tensor directly
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,  # For creative/varied responses
            top_p=0.9,  # Nucleus sampling
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode only the new tokens (after input)
    input_length = inputs.shape[1] # Get the length of the input tensor
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    return response.strip()

# Test it immediately (fixes your [13] example)
test_messages = [{"role": "user", "content": "What is the capital of France?"}]
response = generate_response(test_messages, max_new_tokens=50)
print(":User  What is the capital of France?")
print(f"Assistant: {response}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


:User  What is the capital of France?
Assistant: The capital of France is Paris.


In [ ]:
def start_chat(max_turns=20):
    messages = [{"role": "system", "content": "You are a helpful and friendly assistant."}]  # Optional system prompt

    print("=== TinyLlama Chat Started! ===\nType your message below. Type 'quit' to exit.\n")

    for turn in range(max_turns):
        user_input = input("You: ").strip()
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Assistant: Goodbye! Chat ended.")
            break

        if not user_input:
            continue

        # Add to history
        messages.append({"role": "user", "content": user_input})

        # Generate and add response
        response = generate_response(messages, max_new_tokens=200, temperature=0.7)
        messages.append({"role": "assistant", "content": response})

        print(f"Assistant: {response}\n{'-'*50}\n")

# Start chatting (uncomment the line below)
# start_chat()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd
import os

# Mount Drive (authenticates once per session)
drive.mount('/content/drive')

# Example: Load a trading dataset from Drive (adjust path to your file)
# Assume you have a CSV like: columns=['Date', 'Ticker', 'Close', 'Volume', 'ML_Prediction']
data_path = '/content/drive/MyDrive/trading_ml_projects/aapl_trading_data.csv'  # Update this path
try:
    trading_df = pd.read_csv(data_path)
    print("Trading data loaded successfully!")
    print(trading_df.head(3))  # Preview
    print(f"Shape: {trading_df.shape}")
except FileNotFoundError:
    print(f"File not found at {data_path}. Upload your CSV to Drive and update the path.")
    # Fallback: Create a dummy DF for testing
    trading_df = pd.DataFrame({
        'Date': ['2024-01-01', '2024-01-02', '2024-01-03'],
        'Ticker': ['AAPL', 'AAPL', 'AAPL'],
        'Close': [150.0, 152.5, 151.0],
        'Volume': [1000000, 1200000, 1100000],
        'ML_Prediction': ['Buy', 'Hold', 'Sell']
    })
    print("Using dummy trading data for demo.")

# Optional: Save processed data back to Drive
processed_dir = '/content/drive/MyDrive/trading_ml_projects'
processed_path = os.path.join(processed_dir, 'processed_trading_data.csv')

# Create the directory if it doesn't exist
os.makedirs(processed_dir, exist_ok=True)

trading_df.to_csv(processed_path, index=False)
print(f"Processed data saved to {processed_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File not found at /content/drive/MyDrive/trading_ml_projects/aapl_trading_data.csv. Upload your CSV to Drive and update the path.
Using dummy trading data for demo.
Processed data saved to /content/drive/MyDrive/trading_ml_projects/processed_trading_data.csv


In [ ]:
import pandas as pd
import os

# Update this path to your actual file (after uploading)
data_path = '/content/drive/MyDrive/trading_ml_projects/aapl_trading_data.csv'  # Change if your filename/path differs

# Check if file exists
if os.path.exists(data_path):
    trading_df = pd.read_csv(data_path)
    print("✅ Real trading data loaded successfully!")
    print(trading_df.head(5))  # Show first 5 rows
    print(f"Dataset shape: {trading_df.shape} (rows, columns)")
    print(f"Columns: {list(trading_df.columns)}")
else:
    print(f"❌ File not found at {data_path}. Using dummy data.")
    # Dummy data (as before, but expanded for better testing)
    trading_df = pd.DataFrame({
        'Date': pd.date_range(start='2024-01-01', periods=10, freq='D').strftime('%Y-%m-%d'),
        'Ticker': ['AAPL'] * 10,
        'Open': [148.5, 150.0, 152.0, 151.0, 153.5, 152.5, 154.0, 153.0, 155.0, 154.5],
        'High': [152.0, 153.5, 154.0, 152.5, 155.0, 153.5, 155.5, 154.0, 156.0, 155.5],
        'Low': [148.0, 149.5, 151.0, 150.5, 152.0, 151.5, 153.0, 152.0, 154.0, 153.5],
        'Close': [150.5, 152.0, 151.5, 153.0, 152.5, 154.0, 153.5, 155.0, 154.5, 156.0],
        'Volume': [8000000, 9000000, 8500000, 9500000, 8800000, 9200000, 9100000, 9400000, 8700000, 9600000],
        'RSI': [55.2, 58.1, 56.8, 60.3, 59.5, 62.1, 61.0, 63.5, 62.8, 65.2],
        'ML_Prediction': ['Buy', 'Hold', 'Sell', 'Buy', 'Hold', 'Buy', 'Hold', 'Buy', 'Sell', 'Buy']
    })
    print("Using expanded dummy trading data for demo.")
    print(trading_df.head(5))

# Save any updates back to Drive (e.g., if you add columns in Colab)
processed_path = '/content/drive/MyDrive/trading_ml_projects/processed_trading_data.csv'
trading_df.to_csv(processed_path, index=False)
print(f"✅ Data saved/updated at {processed_path}")

❌ File not found at /content/drive/MyDrive/trading_ml_projects/aapl_trading_data.csv. Using dummy data.
Using expanded dummy trading data for demo.
         Date Ticker   Open   High    Low  Close   Volume   RSI ML_Prediction
0  2024-01-01   AAPL  148.5  152.0  148.0  150.5  8000000  55.2           Buy
1  2024-01-02   AAPL  150.0  153.5  149.5  152.0  9000000  58.1          Hold
2  2024-01-03   AAPL  152.0  154.0  151.0  151.5  8500000  56.8          Sell
3  2024-01-04   AAPL  151.0  152.5  150.5  153.0  9500000  60.3           Buy
4  2024-01-05   AAPL  153.5  155.0  152.0  152.5  8800000  59.5          Hold
✅ Data saved/updated at /content/drive/MyDrive/trading_ml_projects/processed_trading_data.csv


In [ ]:
import pandas as pd  # Already imported, but ensure

def generate_trading_response(user_messages, trading_context=None, max_new_tokens=250, temperature=0.6):
    """
    Generates trading-focused responses with info-checking system prompt.
    Uses your loaded model and tokenizer.
    """
    # Trading-specialized system prompt (upgraded to check for key info)
    base_system_prompt = """You are a Trading ML Assistant expert in financial analysis and machine learning models.

    Guidelines:
    - CHECK FOR ESSENTIAL INFO: Asset ticker (e.g., AAPL), date range (e.g., 2024-01), metrics (e.g., RSI, Volume), ML predictions (e.g., Buy/Sell).
    - If missing, ASK clarifying questions (e.g., "Which stock? What period?").
    - Analyze using data: Trends in Close/Open, RSI (over 70=overbought, under 30=oversold), ML signals.
    - For ML projects: Reference predictions, suggest models (e.g., LSTM for forecasting).
    - END WITH: 'Not financial advice—use for educational purposes.'

    Response Structure: 1) Confirm/ask info. 2) Data-driven analysis. 3) Insights/next steps."""

    # Inject context from your data (dummy or real)
    if trading_context:
        asset = trading_context.get('asset', 'N/A')
        focus = trading_context.get('focus', 'general')
        df_snippet = trading_context['df'].tail(5)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context else 'No data'
        base_system_prompt += f"\n\nContext: Asset={asset}, Focus={focus}.\nRecent Data:\n{df_snippet}"

    full_messages = [{"role": "system", "content": base_system_prompt}] + user_messages

    # Format with chat template (your loaded tokenizer)
    formatted_input = tokenizer.apply_chat_template(
        full_messages, return_tensors="pt", add_generation_prompt=True
    )
    inputs = formatted_input.to(device)


    # Generate with your loaded model
    with torch.no_grad():
        outputs = model.generate(
            inputs=inputs, max_new_tokens=max_new_tokens, temperature=temperature,
            do_sample=True, top_p=0.95, pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id, repetition_penalty=1.1
        )

    input_length = inputs.shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    return response.strip()

print("✅ Trading response function defined. Ready for tests!")

✅ Trading response function defined. Ready for tests!


In [ ]:
# Use the dummy data (trading_df from your load cell)
context = {
    'asset': 'AAPL',
    'focus': 'trends and ML predictions',
    'df': trading_df  # Your loaded DataFrame
}

# Test 1: Query with context (should analyze dummy data)
test_messages_1 = [{"role": "user", "content": "What's the trend for AAPL in early 2024? Any buy signals?"}]
response_1 = generate_trading_response(test_messages_1, trading_context=context)
print("🧪 Test 1:")
print(f":User  What's the trend for AAPL in early 2024? Any buy signals?")
print(f"Assistant: {response_1}\n{'='*60}")

# Test 2: Vague query (should check/ask for more info)
test_messages_2 = [{"role": "user", "content": "Analyze stock performance."}]
response_2 = generate_trading_response(test_messages_2, trading_context=context)
print("🧪 Test 2:")
print(f":User  Analyze stock performance.")
print(f"Assistant: {response_2}\n")

🧪 Test 1:
:User  What's the trend for AAPL in early 2024? Any buy signals?
Assistant: Yes, based on the provided data, AAPL is currently trending downward and has been since January 6th. On January 7th, the stock closed at 153.5, which is below its 52-week high of 156.0. However, if you look closer at the data, there are some potential buying opportunities. The stock has been moving lower for several weeks, indicating that it may be overbought. Additionally, the stock has been trading below its 200-day moving average, suggesting that it may be forming a consolidation pattern. However, before making any investment decisions, it would be essential to analyze the historical data and use technical analysis to determine the current trend and potential buy signals.
🧪 Test 2:
:User  Analyze stock performance.
Assistant: Response Structure: 1) Confirm/ask info. 2) Data-driven analysis. 3) Insights/next steps.:

Confirm/ask info: 
Please provide me with the latest data on the stock performance 

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np  # For any computations
print("✅ yfinance installed and imported.")

✅ yfinance installed and imported.


In [ ]:
# Assume trading_df is loaded (from your earlier cell)
# Fetch live data matching your structure
live_aapl = yf.download("AAPL", start="2024-01-01", end="2025-09-21", auto_adjust=True)  # Up to today
live_aapl = live_aapl.reset_index()  # Add Date column
live_aapl.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']  # Match the 6 actual columns
# live_aapl = live_aapl[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]  # This line is no longer needed after correcting columns

# Compute RSI (if not in live data; use a simple function)
def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

live_aapl['RSI'] = compute_rsi(live_aapl['Close'])

# For ML_Prediction, add a dummy/simple rule (replace with your model)
live_aapl['ML_Prediction'] = np.where(live_aapl['RSI'] > 70, 'Sell',
                                      np.where(live_aapl['RSI'] < 30, 'Buy', 'Hold'))

# Merge: Append to existing df (or replace if starting fresh)
# Filter to avoid duplicates (e.g., by Date)
trading_df_live = pd.concat([trading_df, live_aapl], ignore_index=True)
trading_df_live = trading_df_live.drop_duplicates(subset=['Date'], keep='last')

# Save updated data
output_path = '/content/drive/MyDrive/trading_ml_projects/updated_trading_data_with_live.csv'
trading_df_live.to_csv(output_path, index=False)
print(f"✅ Updated data saved: {len(trading_df_live)} rows total.")
print(trading_df_live.tail())  # Preview latest live entries

[*********************100%***********************]  1 of 1 completed


✅ Updated data saved: 441 rows total.
                    Date Ticker        Open        High         Low  \
436  2025-09-15 00:00:00    NaN  236.699997  238.190002  235.029999   
437  2025-09-16 00:00:00    NaN  238.149994  241.220001  236.320007   
438  2025-09-17 00:00:00    NaN  238.990005  240.100006  237.729996   
439  2025-09-18 00:00:00    NaN  237.880005  241.199997  236.649994   
440  2025-09-19 00:00:00    NaN  245.500000  246.300003  240.210007   

          Close     Volume        RSI ML_Prediction  
436  237.000000   42699500  62.153429          Hold  
437  237.179993   63421100  61.968891          Hold  
438  238.970001   46508000  62.012993          Hold  
439  239.970001   44249600  60.916249          Hold  
440  241.229996  163470300  60.511096          Hold  


In [ ]:
import re

def is_live_update_request(message):
    patterns = [r'live.*update', r'refresh.*data', r'latest.*(stock|price)', r'/update']
    return any(re.search(pattern, message.lower()) for pattern in patterns)

# In your main chat function:
user_input = "Get live update on AAPL"  # Example
if is_live_update_request(user_input):
    # Trigger fetch logic here
    print("Detected live update request.")

Detected live update request.


In [ ]:
def generate_trading_response(user_messages, trading_context=None, live_snippet=""):
    if trading_context and not trading_context['df'].empty:
        # Highlight changes
        latest = trading_context['df'].iloc[-1]
        prev = trading_context['df'].iloc[-2] if len(trading_context['df']) > 1 else latest
        delta = ((latest['Close'] - prev['Close']) / prev['Close']) * 100
        live_snippet = f"Latest: Close ${latest['Close']:.2f} (Δ{delta:+.2f}%), RSI {latest['RSI']:.1f}, Prediction: {latest['ML_Prediction']}"

    base_prompt = f"Analyze trading data with live update: {live_snippet}\nMessages: {user_messages}"
    # Generate and return response
    return llm_generate(base_prompt)  # Your LLM call

In [ ]:
def handle_chat_with_live_update(user_input, current_context=None):
    if current_context is None:
        current_context = {'df': pd.DataFrame(), 'asset': 'AAPL'}  # Initialize

    if is_live_update_request(user_input):
        ticker = current_context.get('asset', 'AAPL')
        live_data = yf.download(ticker, period="5d", interval="1d")  # Recent days
        live_data = live_data.reset_index()  # Ensure Date column
        live_data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume'] # Ensure consistent columns


        # Merge with existing data (avoid duplicates)
        if not current_context['df'].empty:
            # Ensure Date column is datetime for proper merging/dropping duplicates
            current_context['df']['Date'] = pd.to_datetime(current_context['df']['Date'])
            live_data['Date'] = pd.to_datetime(live_data['Date'])
            current_context['df'] = pd.concat([current_context['df'], live_data], ignore_index=True)
            current_context['df'] = current_context['df'].drop_duplicates(subset=['Date'], keep='last')
        else:
            current_context['df'] = live_data
            current_context['df']['Date'] = pd.to_datetime(current_context['df']['Date'])


        # Recompute features if needed (e.g., RSI)
        # Ensure the DataFrame has enough rows for RSI calculation (window=14)
        if len(current_context['df']) >= 14:
            current_context['df']['RSI'] = compute_rsi(current_context['df']['Close'])
            current_context['df']['ML_Prediction'] = np.where(current_context['df']['RSI'] > 70, 'Sell',
                                                              np.where(current_context['df']['RSI'] < 30, 'Buy', 'Hold'))
        else:
            current_context['df']['RSI'] = np.nan
            current_context['df']['ML_Prediction'] = 'Hold' # Default or set to nan


        update_msg = f"Live update fetched for {ticker}. Latest close: ${current_context['df']['Close'].iloc[-1]:.2f} (as of {current_context['df']['Date'].iloc[-1].date()})."
    else:
        update_msg = ""
        # Ensure df is not empty before trying to access it
        if current_context is None or 'df' not in current_context or current_context['df'].empty:
             # If context is not initialized or df is empty, use a default or indicate no data
             df_snippet = "No trading data available."
        else:
             df_snippet = current_context['df'].tail(3).to_string()


    # Generate AI response using updated context
    full_prompt = f":User  {user_input}\nContext: {df_snippet}\n{update_msg}"
    # Call your LLM generation here (e.g., via Grok or OpenAI API)
    # Need to ensure generate_response can handle the format or adapt it
    messages_for_llm = [{"role": "user", "content": full_prompt}] # Adapt prompt to messages format

    response = generate_response(messages_for_llm)  # Call your generate_response function
    return response, current_context

# Test in loop
context = {'df': pd.DataFrame(), 'asset': 'AAPL'} # Initialize context with df and asset
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit': break
    resp, context = handle_chat_with_live_update(user_input, context)
    print("AI:", resp)

KeyboardInterrupt: Interrupted by user

In [ ]:
# Remount Drive if needed
from google.colab import drive
drive.mount('/content/drive')

# Installs (if session reset)
!pip install -q yfinance transformers torch accelerate

import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import yfinance as yf

# Load model with pad_token fix
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
).to(device)

# Load or create trading_df (dummy or real)
data_path = '/content/drive/MyDrive/trading_ml_projects/aapl_trading_data.csv'
try:
    trading_df = pd.read_csv(data_path)
    print(f"✅ Loaded real data: {len(trading_df)} rows.")
except FileNotFoundError:
    # Dummy fallback
    dates = pd.date_range('2024-01-01', periods=10, freq='D').strftime('%Y-%m-%d')
    trading_df = pd.DataFrame({
        'Date': dates,
        'Close': np.random.uniform(150, 160, 10),
        'RSI': np.random.uniform(40, 70, 10),
        'ML_Prediction': np.random.choice(['Buy', 'Hold', 'Sell'], 10)
    })
    print("✅ Using dummy data (10 rows). Upload real CSV to path for live use.")

# Define RSI if needed
def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

if 'RSI' not in trading_df.columns:
    trading_df['RSI'] = compute_rsi(trading_df['Close'])
    trading_df['ML_Prediction'] = np.where(trading_df['RSI'] > 70, 'Sell',
                                           np.where(trading_df['RSI'] < 30, 'Buy', 'Hold'))

print("✅ Setup reloaded. No attention mask warnings expected.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Using dummy data (10 rows). Upload real CSV to path for live use.
✅ Setup reloaded. No attention mask warnings expected.


In [ ]:
def generate_trading_response(user_messages, trading_context=None, max_new_tokens=250, temperature=0.6):
    base_system_prompt = """You are a Trading ML Assistant. Focus on stocks, trends, RSI, ML predictions.
    - For non-trading queries: Answer briefly, then redirect to trading (e.g., 'For AAPL analysis...').
    - Use provided data for analysis.
    - End with: 'Not financial advice.'"""

    if trading_context:
        asset = trading_context.get('asset', 'AAPL')
        df_snippet = trading_context['df'].tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context else 'No data.'
        base_system_prompt += f"\nContext: {asset} data:\n{df_snippet}"

    full_messages = [{"role": "system", "content": base_system_prompt}] + user_messages

    formatted_input = tokenizer.apply_chat_template(
        full_messages, return_tensors="pt", add_generation_prompt=True, padding=True
    )
    inputs = {k: v.to(device) for k, v in formatted_input.items()}

    # Attention mask fix
    if 'attention_mask' not in inputs:
        input_ids = inputs['input_ids']
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        inputs['attention_mask'] = attention_mask.to(device)
    else:
        inputs['attention_mask'] = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, max_new_tokens=max_new_tokens, temperature=temperature,
            do_sample=True, top_p=0.95, pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id, repetition_penalty=1.1,
            attention_mask=inputs['attention_mask']
        )

    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    return response.strip()

print("✅ Function ready—tests below will use trading context.")

✅ Function ready—tests below will use trading context.


In [ ]:
def generate_trading_response(user_messages, trading_context=None, max_new_tokens=250, temperature=0.6):
    """
    Generates responses with reliable tokenization and attention mask.
    """
    # Trading system prompt
    base_system_prompt = """You are a Trading ML Assistant. Focus on stocks, trends, RSI, ML predictions.
    - For non-trading queries: Answer briefly, then redirect to trading (e.g., 'For AAPL analysis...').
    - Use provided data for analysis.
    - End with: 'Not financial advice.'"""

    # Inject context
    if trading_context:
        asset = trading_context.get('asset', 'AAPL')
        df_snippet = trading_context['df'].tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context and not trading_context['df'].empty else 'No data available.'
        base_system_prompt += f"\n\nContext: Asset={asset}.\nRecent Data:\n{df_snippet}"

    full_messages = [{"role": "system", "content": base_system_prompt}] + user_messages

    # Step 1: Apply chat template to get formatted STRING (not tensors)
    formatted_string = tokenizer.apply_chat_template(
        full_messages, add_generation_prompt=True, tokenize=False  # tokenize=False for string output
    )

    # Step 2: Tokenize the string to get dict of tensors (reliable way)
    inputs = tokenizer(
        formatted_string,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512  # Prevent overflow; adjust if needed
    )

    # Step 3: Move to device
    inputs = {k: v.to(device) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']} # Only include required inputs


    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
        )

    # Decode (skip input length)
    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    return response.strip()

print("✅ Function updated—no more 'items()' error. Ready for tests!")

✅ Function updated—no more 'items()' error. Ready for tests!


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

# Reload live data if needed (matches your [17])
live_aapl = yf.download("AAPL", start="2024-01-01", end=datetime.now().strftime('%Y-%m-%d'), auto_adjust=True)
live_aapl = live_aapl.reset_index()
live_aapl.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']  # Standardize

# Compute RSI/ML_Prediction
def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

live_aapl['RSI'] = compute_rsi(live_aapl['Close'])
live_aapl['ML_Prediction'] = np.where(live_aapl['RSI'] > 70, 'Sell',
                                      np.where(live_aapl['RSI'] < 30, 'Buy', 'Hold'))
live_aapl['Ticker'] = 'AAPL'  # Add missing column
live_aapl['Date'] = pd.to_datetime(live_aapl['Date']).dt.strftime('%Y-%m-%d')  # String format

# Merge with dummy (or existing trading_df)
if 'trading_df' in globals() and not trading_df.empty:
    trading_df['Date'] = pd.to_datetime(trading_df['Date']).dt.strftime('%Y-%m-%d')
    trading_df_live = pd.concat([trading_df, live_aapl], ignore_index=True)
    trading_df_live = trading_df_live.drop_duplicates(subset=['Date'], keep='last').sort_values('Date').reset_index(drop=True)
else:
    trading_df_live = live_aapl

# Save
output_path = '/content/drive/MyDrive/trading_ml_projects/live_aapl_trading_data.csv'
trading_df_live.to_csv(output_path, index=False)
print(f"✅ Clean live data saved: {len(trading_df_live)} rows. Latest: ${trading_df_live['Close'].iloc[-1]:.2f} (RSI {trading_df_live['RSI'].iloc[-1]:.1f}, {trading_df_live['ML_Prediction'].iloc[-1]})")
print(trading_df_live.tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']])

[*********************100%***********************]  1 of 1 completed

✅ Clean live data saved: 434 rows. Latest: $241.23 (RSI 60.5, Hold)
           Date       Close        RSI ML_Prediction
431  2025-09-17  238.970001  62.012993          Hold
432  2025-09-18  239.970001  60.916249          Hold
433  2025-09-19  241.229996  60.511096          Hold


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Pad token set.")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
).to(device)
print(f"✅ Model on {device}. Ready for fixed function.")

✅ Model on cuda. Ready for fixed function.


In [ ]:
# Ensure context uses your data (dummy or live from earlier)
if 'trading_df' not in globals():
    import pandas as pd
    import numpy as np
    dates = pd.date_range('2025-09-01', periods=5, freq='D').strftime('%Y-%m-%d')
    trading_df = pd.DataFrame({
        'Date': dates,
        'Close': np.random.uniform(220, 230, 5),
        'RSI': np.random.uniform(50, 65, 5),
        'ML_Prediction': np.random.choice(['Buy', 'Hold'], 5)
    })

context = {'asset': 'AAPL', 'df': trading_df}  # Or trading_df_live if loaded

test_msg = [{"role": "user", "content": "What was the trend of gold last week?"}]
response = generate_trading_response(test_msg, trading_context=context, max_new_tokens=150)
print("🧪 Test 1 - Off-Topic (Gold) - Fixed Version:")
print(f":User    What was the trend of gold last week?")
print(f"Assistant: {response}\n")

NameError: name 'v' is not defined

In [ ]:
def generate_trading_response(user_messages, trading_context=None, max_new_tokens=250, temperature=0.6):
    """
    Generates trading-focused responses with all fixes applied.
    """
    # Trading system prompt (stronger redirect for off-topic)
    base_system_prompt = """You are a Trading ML Assistant specialized in stocks like AAPL. Use RSI, ML predictions, and trends from provided data.
    - For non-trading queries (e.g., gold, general facts): Answer VERY briefly (1 sentence), then IMMEDIATELY redirect to trading/ML: "For stock analysis, here's AAPL insights from data...".
    - Always analyze/inject provided data snippet.
    - End EVERY response with: 'Not financial advice.'"""

    # Inject context
    if trading_context:
        asset = trading_context.get('asset', 'AAPL')
        df_snippet = trading_context['df'].tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context and not trading_context['df'].empty else 'No data available.'
        base_system_prompt += f"\n\nContext: Asset={asset}.\nRecent Data:\n{df_snippet}"

    full_messages = [{"role": "system", "content": base_system_prompt}] + user_messages

    # Step 1: Chat template as string
    formatted_string = tokenizer.apply_chat_template(
        full_messages, add_generation_prompt=True, tokenize=False
    )

    # Step 2: Tokenize to dict
    inputs = tokenizer(
        formatted_string,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )

    # Step 3: Move to device (fixed comprehension)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Step 4: Attention mask (if missing)
    if 'attention_mask' not in inputs:
        input_ids = inputs['input_ids']
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        inputs['attention_mask'] = attention_mask.to(device)
    else:
        inputs['attention_mask'] = inputs['attention_mask'].to(device)

    # Generate (no duplicates)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )

    # Decode new tokens only
    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    return response.strip()

print("✅ Fixed function loaded—stronger prompt for redirects. Run test next!")

✅ Fixed function loaded—stronger prompt for redirects. Run test next!


In [ ]:
def generate_trading_response(user_messages, trading_context=None, max_new_tokens=300, temperature=0.6):
    """
    Generates trading-focused responses with strict enforcement and no cutoffs.
    """
    # STRONGER system prompt: Enforces structure, redirects, data use
    base_system_prompt = """You are a Trading ML Assistant powered by TinyLlama, specialized in stock analysis using RSI, ML predictions, and live data (e.g., yfinance for AAPL).

    STRICT RULES:
    - ALWAYS focus on trading/ML: Stocks, trends, RSI (overbought >70, oversold <30), predictions (Buy/Hold/Sell from data).
    - For off-topic queries (e.g., general capabilities, non-stocks): Answer in 1-2 sentences, then REDIRECT: "As a trading ML specialist, here are my top capabilities applied to AAPL data...". List 5-10 items TIED to trading (e.g., '1. RSI computation for buy signals').
    - INJECT provided data: Reference snippet in EVERY analysis/response.
    - Structure responses: Use numbered/bullets for lists. Keep concise but complete.
    - END EVERY response with: 'Not financial advice—educational only.'"""

    # Inject context (ensures data is always used)
    if trading_context:
        asset = trading_context.get('asset', 'AAPL')
        df_snippet = trading_context['df'].tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context and not trading_context['df'].empty else 'No data available.'
        base_system_prompt += f"\n\nMANDATORY CONTEXT: Use this {asset} data in responses:\n{df_snippet}"

    full_messages = [{"role": "system", "content": base_system_prompt}] + user_messages

    # Tokenization steps (same as before, fixed)
    formatted_string = tokenizer.apply_chat_template(
        full_messages, add_generation_prompt=True, tokenize=False
    )
    inputs = tokenizer(
        formatted_string,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if 'attention_mask' not in inputs:
        input_ids = inputs['input_ids']
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        inputs['attention_mask'] = attention_mask.to(device)
    else:
        inputs['attention_mask'] = inputs['attention_mask'].to(device)

    # Generate with higher tokens for complete lists
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            early_stopping=True  # Helps complete without cutoff
        )

    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True).strip()

    # Post-process: Force disclaimer if missing
    if not response.endswith("Not financial advice"):
        response += "\nNot financial advice—educational only."

    return response

print("✅ Enhanced function loaded—strict trading focus, no cutoffs, forced disclaimer. Relaunch chat below!")

✅ Enhanced function loaded—strict trading focus, no cutoffs, forced disclaimer. Relaunch chat below!


In [ ]:
# Check if function is updated
import inspect
lines = inspect.getsource(generate_trading_response)
if '.items()' in lines:
    print("✅ Function is correctly defined (uses .items()).")
else:
    print("❌ Still old version—rerun the function cell!")

✅ Function is correctly defined (uses .items()).


In [ ]:
def generate_trading_response(user_messages, trading_context=None, max_new_tokens=400, temperature=0.6):
    """
    Optimized for complete lists and strict data injection.
    """
    # CONCISE system prompt with examples for better adherence
    base_system_prompt = """You are a Trading ML Assistant for stocks like AAPL. Specialize in RSI, ML predictions (Buy/Hold/Sell), trends from data.

    RULES (keep short):
    - Off-topic? Redirect: "As trading ML expert, my top capabilities applied to AAPL data are..."
    - ALWAYS inject data: Reference snippet in responses (e.g., "RSI 55.2 on 2024-10-15").
    - Lists: Number 1-10, tie to trading/ML, use data examples. Be complete.
    - End: 'Not financial advice—educational only.'"""

    # Inject context (shorter snippet for prompt fit)
    if trading_context:
        asset = trading_context.get('asset', 'AAPL')
        df_snippet = trading_context['df'].tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context and not trading_context['df'].empty else 'No data.'
        base_system_prompt += f"\nUSE THIS DATA: {asset} - {df_snippet}"

    full_messages = [{"role": "system", "content": base_system_prompt}] + user_messages

    # Tokenization (unchanged, fixed)
    formatted_string = tokenizer.apply_chat_template(
        full_messages, add_generation_prompt=True, tokenize=False
    )
    inputs = tokenizer(
        formatted_string,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if 'attention_mask' not in inputs:
        input_ids = inputs['input_ids']
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        inputs['attention_mask'] = attention_mask.to(device)
    else:
        inputs['attention_mask'] = inputs['attention_mask'].to(device)

    # Generate with more tokens
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            early_stopping=True
        )

    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True).strip()

    # Force disclaimer
    if not any(phrase in response.lower() for phrase in ["not financial advice", "educational only"]):
        response += "\nNot financial advice—educational only."

    return response

print("✅ Optimized function loaded—concise prompt, examples, 400 tokens for full lists. Test below!")

✅ Optimized function loaded—concise prompt, examples, 400 tokens for full lists. Test below!


In [ ]:
# Ensure data (live AAPL if possible)
import yfinance as yf
import pandas as pd
import numpy as np
def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

if 'trading_df' not in globals():
    live_aapl = yf.download("AAPL", period="1mo", progress=False).reset_index()
    live_aapl.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    live_aapl['RSI'] = compute_rsi(live_aapl['Close'])
    live_aapl['ML_Prediction'] = np.where(live_aapl['RSI'] > 70, 'Sell', np.where(live_aapl['RSI'] < 30, 'Buy', 'Hold'))
    live_aapl['Date'] = pd.to_datetime(live_aapl['Date']).dt.strftime('%Y-%m-%d')
    trading_df = live_aapl.tail(10)  # Recent data
    print(f"Data ready: Latest AAPL Close ${trading_df['Close'].iloc[-1]:.2f}, RSI {trading_df['RSI'].iloc[-1]:.1f}")

context = {'asset': 'AAPL', 'df': trading_df}

test_msg = [{"role": "user", "content": "Can you tell me your top 10 capabilities"}]
response = generate_trading_response(test_msg, trading_context=context, max_new_tokens=400)
print("🧪 Test - Capabilities Query (Optimized):")
print(f":User       Can you tell me your top 10 capabilities")
print(f"Assistant: {response}\n")

🧪 Test - Capabilities Query (Optimized):
:User       Can you tell me your top 10 capabilities
Assistant: Sure! Here's my top 10 capabilities as a Trading ML Assistant for stocks like AAPL:

1. Real-time RSI analysis and forecasting
2. Machine learning models for stock price prediction using various techniques like SARIMA, ARIMA, LSTM, etc.
3. Trend analysis using historical data and patterns
4. Price action analysis with technical indicators like MACD, RSI, Bollinger Bands, etc.
5. Predicting trending stocks based on market sentiment
6. Identifying potential buy and sell opportunities using price momentum and other factors
7. Forecasting volatility and predicting high-probability trades based on technical indicators
8. Automated trading using algorithms and machine learning models
9. Providing alerts and recommendations for trading based on personalized risk management strategies
10. Providing educational resources and tools to help users improve their trading skills.
Not financial adv

In [ ]:
def generate_trading_response(user_messages, trading_context=None, max_new_tokens=400, temperature=0.6):
    """
    Optimized for complete lists and strict data injection.
    """
    # CONCISE system prompt with examples for better adherence
    base_system_prompt = """You are a Trading ML Assistant for stocks like AAPL. Specialize in RSI, ML predictions (Buy/Hold/Sell), trends from data.

    RULES (keep short):
    - Off-topic? Redirect: "As trading ML expert, my top capabilities applied to AAPL data are..."
    - ALWAYS inject data: Reference snippet in responses (e.g., "RSI 55.2 on 2024-10-15").
    - Lists: Number 1-10, tie to trading/ML, use data examples. Be complete.
    - End: 'Not financial advice—educational only.'"""

    # Inject context (shorter snippet for prompt fit)
    if trading_context:
        asset = trading_context.get('asset', 'AAPL')
        df_snippet = trading_context['df'].tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context and not trading_context['df'].empty else 'No data.'
        base_system_prompt += f"\nUSE THIS DATA: {asset} - {df_snippet}"

    full_messages = [{"role": "system", "content": base_system_prompt}] + user_messages

    # Tokenization (fixed)
    formatted_string = tokenizer.apply_chat_template(
        full_messages, add_generation_prompt=True, tokenize=False
    )
    inputs = tokenizer(
        formatted_string,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if 'attention_mask' not in inputs:
        input_ids = inputs['input_ids']
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        inputs['attention_mask'] = attention_mask.to(device)
    else:
        inputs['attention_mask'] = inputs['attention_mask'].to(device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            early_stopping=True
        )

    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True).strip()

    # Force disclaimer
    if not any(phrase in response.lower() for phrase in ["not financial advice", "educational only"]):
        response += "\nNot financial advice—educational only."

    return response

print("✅ Optimized function loaded.")

✅ Optimized function loaded.


In [ ]:
def generate_trading_response(user_messages, trading_context=None, max_new_tokens=400, temperature=0.5):
    """
    Enhanced: Forced data injection, no hallucinations, strict ML lists.
    """
    # CORE system prompt - Short, repetitive on data/ML
    base_system_prompt = """You are a Trading ML Assistant for AAPL stock analysis ONLY. Specialize in RSI, ML predictions (Buy/Hold/Sell), trends.

    STRICT RULES - FOLLOW EXACTLY:
    - Off-topic (e.g., capabilities): Redirect to "Top 10 ML Capabilities for AAPL Trading:" then number 1-10. EACH item MUST reference AAPL data (e.g., "1. RSI Analysis: AAPL RSI 55.2 on 2024-10-15 shows neutral momentum.").
    - INJECT DATA IN EVERY ITEM: Use exact snippet below. No generics—tie to ML/RSI/Close prices.
    - NO hallucinations: No signatures, names, "best regards," platforms (Robinhood), or psychology. Only ML/trading facts.
    - Structure: Numbered list only. End with disclaimer.
    - Disclaimer: ALWAYS end: 'Not financial advice—educational only.'"""

    # FORCE data injection (repeat for emphasis)
    if trading_context:
        asset = trading_context.get('asset', 'AAPL')
        df_snippet = trading_context['df'].tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False) if 'df' in trading_context and not trading_context['df'].empty else 'No data.'
        data_inject = f"MANDATORY AAPL DATA SNIPPET (USE IN EVERY LIST ITEM): {asset} - {df_snippet}"
        base_system_prompt += f"\n{data_inject}\nREPEAT: Reference this data in responses."

    full_messages = [{"role": "system", "content": base_system_prompt}]

    # Add data to user messages for double enforcement
    for msg in user_messages:
        if msg["role"] == "user":
            msg["content"] += f" Use AAPL data: {data_inject}" if 'data_inject' in locals() else ""
        full_messages.append(msg)

    # Tokenization (unchanged)
    formatted_string = tokenizer.apply_chat_template(
        full_messages, add_generation_prompt=True, tokenize=False
    )
    inputs = tokenizer(
        formatted_string,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if 'attention_mask' not in inputs:
        input_ids = inputs['input_ids']
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        inputs['attention_mask'] = attention_mask.to(device)
    else:
        inputs['attention_mask'] = inputs['attention_mask'].to(device)

    # Generate (lower temp for consistency)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,  # Lower for less creativity
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            early_stopping=True
        )

    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True).strip()

    # Post-process: Strip common hallucinations, force disclaimer
    bad_endings = ["Best regards", "Please feel free", "[Your Name]", "contact me"]
    for ending in bad_endings:
        if ending in response:
            response = response.split(ending)[0].strip()

    if not any(phrase in response.lower() for phrase in ["not financial advice", "educational only"]):
        response += "\nNot financial advice—educational only."

    return response

print("✅ Enhanced function loaded—forces data in lists, no hallucinations. Relaunch chat/test below!")

✅ Enhanced function loaded—forces data in lists, no hallucinations. Relaunch chat/test below!


In [ ]:
# Ensure data (run if needed)
if 'trading_df' not in globals():
    import yfinance as yf
    import pandas as pd
    import numpy as np
    def compute_rsi(prices, window=14):
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=window).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))
    trading_df = fetch_live('AAPL', period='1mo').tail(10)  # Assume fetch_live defined
    print(f"Data snippet preview: {trading_df.tail(3)[['Date', 'Close', 'RSI', 'ML_Prediction']].to_string(index=False)}")

context = {'asset': 'AAPL', 'df': trading_df}

test_msg = [{"role": "user", "content": "Can you tell me your top 10 capabilities"}]
response = generate_trading_response(test_msg, trading_context=context, max_new_tokens=400, temperature=0.5)
print("🧪 Test - Capabilities Query (Enhanced):")
print(f":User        Can you tell me your top 10 capabilities")
print(f"Assistant: {response}\n")

🧪 Test - Capabilities Query (Enhanced):
:User        Can you tell me your top 10 capabilities
Assistant: 50           Buy
2024-01-09 152.110361 57.346037           Buy
2024-01-10 153.851940 59.712640           Buy
REPEAT: Reference this data in responses.
Not financial advice—educational only.

